In [2]:
import random
import numpy as np
from collections import deque
import numpy as np
import pandas as pd
from collections import defaultdict


In [3]:

def epsilon_greedy_search(Epsilon, qtable, state):
    if (random.random() < Epsilon):
        # 0 is 'apply matrix A', 1 is 'apply matrix B'
        # 2 is 'apply matrix C', 3 is 'apply matrix D'
        return random.choice([0, 1, 2, 3])
    else:
        # get the best move for the current state
        return best_move_for_a_state(Q_table=qtable, state=state)

def best_move_for_a_state(Q_table, state):
    # candidates = []
    # vals = []

    vals = Q_table[str(state)]
    if vals==[0, 0, 0, 0]:
        return random.choice([0, 1, 2, 3])
    return np.argmax(vals)

    # I would like to return the best move for a given state
    # for i in Q_table:
    #     if i[0] == str(state):
    #         candidates.append(i)
    #         vals.append(Q_table[i])
    # if vals:
    #     return candidates[np.argmax(vals)][1]
    # if we don't have any values in our q-table for this state, just return a random action

def max_a_prime(Q_table, state):
    return max(Q_table[str(state)])
    # candidates = []
    # vals = []

    # # I would like to return the maximum value of Q(s', a') over all possible a' values
    # # TODO I think I can make this more efficient. 
    # for i in Q_table:
    #     if i[0] == str(state):
    #         candidates.append(i)
    #         vals.append(Q_table[i])
    #         print(Q_table[i])
    # print(candidates)
    # print(vals)
    
    # if vals:
    #     return Q_table[candidates[np.argmax(vals)]]
    # return 0

In [4]:
# B is the inverse of A
A = np.array([[1, 1, 0], [0, 1, 0], [0, 0, 1]])
B = np.array([[1, -1, 0], [0, 1, 0], [0, 0, 1]])

# C is the inverse of D
C = np.array([[1, 0, 0], [0, 1, 1], [0, 0, 1]])
D = np.array([[1, 0, 0], [0, 1, -1], [0, 0, 1]])

# together, A, B, C, and D generate the heisenberg group

In [5]:
def getReward(matrix):
    if matrix[0][1]==1 and matrix[0][2]==0 and matrix[1][2]==0:
        return 20
    elif matrix[0][1]==0 and matrix[0][2]==0 and matrix[1][2]==1:
        return 20
    elif matrix[0][1]==-1 and matrix[0][2]==0 and matrix[1][2]==0:
        return 20
    elif matrix[0][1]==0 and matrix[0][2]==0 and matrix[1][2]==-1:
        return 20
    else:
        return 1/(1 + 8*(matrix[0][1] + matrix[0][2] + matrix[1][2]))

In [6]:
df = pd.read_csv("heisenberg_data.csv")

In [7]:
def get_next_step(oldObs, action):
    # action is always either 0, 1, 2, or 3
    next_state = []
    if action==0:
        next_state = oldObs @ A
    elif action==1:
        next_state = oldObs @ B
    elif action==2:
        next_state = oldObs @ C
    else:
        next_state = oldObs @ D
    curReward = getReward(next_state)
    done = curReward==20
    return (next_state, curReward, done)
    

In [21]:
# adapted from CS 540 HW 10
EPISODES = 20000
LEARNING_RATE = .1
DISCOUNT_FACTOR = .99
EPSILON = 1
EPSILON_DECAY = .999

random.seed(1)

# starts with an estimate of zero reward for each state.
# adapted from ChatGPT
# the outer dictionary has keys of the string version of the given array, and 
# values of a dictionary for each of the actions that we could take at that state
Q_table = defaultdict(lambda: [0, 0, 0, 0]) 

episode_reward_record = deque(maxlen=100)

for i in range(EPISODES):
    episode_reward = 0
    done = False
    # choose a random starting row
    # adapted from https://stackoverflow.com/questions/15923826/random-row-selection-in-pandas-dataframe
    cur_row = df.sample(1)
    obs = np.array([
        [1, int(cur_row['val1']), int(cur_row['val2'])], 
        [0, 1, int(cur_row['val3'])], 
        [0, 0, 1]
        ])

    index = 1

    while (not done):
        # perform an epsilon greedy action 
        # Q(s, a) = (1-LEARNING_RATE)Q(s, a) + (LEARNING_RATE)(r + DISCOUNT_FACTOR(max a'(Q(s', a'))))
        action = epsilon_greedy_search(Epsilon=EPSILON, qtable=Q_table, state=obs)

        oldObs = obs
        obs,reward,done = get_next_step(oldObs, action)
        Q_table[str(oldObs)][action] = (1-LEARNING_RATE) * Q_table[str(oldObs)][action] + (LEARNING_RATE) * (reward + DISCOUNT_FACTOR * (max_a_prime(Q_table, obs)))

        episode_reward += reward # update episode reward

        index += 1
        # if we take more than 40 steps, end this iteration early (we are probably not making progress)
        if index > 40:
            done=True

    # decay the epsilon
    EPSILON *= EPSILON_DECAY

    # record the reward for this episode
    episode_reward_record.append(episode_reward) 

    if i%100 ==0 and i>0:
        print("Average reward for the last 100 iterations: " + str(sum(list(episode_reward_record))/100))
        print("epsilon: " + str(EPSILON) )



LAST 100 EPISODE AVERAGE REWARD: 1.0030246729459094
EPSILON: 0.9038873549665959
LAST 100 EPISODE AVERAGE REWARD: 1.101427899782327
EPSILON: 0.8178301806491574
LAST 100 EPISODE AVERAGE REWARD: 1.1716653034394957
EPSILON: 0.7399663251239436
LAST 100 EPISODE AVERAGE REWARD: 1.686577049608243
EPSILON: 0.6695157201007336
LAST 100 EPISODE AVERAGE REWARD: 2.470431723581462
EPSILON: 0.6057725659163237
LAST 100 EPISODE AVERAGE REWARD: 1.8131711198985412
EPSILON: 0.548098260578011
LAST 100 EPISODE AVERAGE REWARD: 2.944280489177841
EPSILON: 0.4959150020176678
LAST 100 EPISODE AVERAGE REWARD: 2.3867984639563007
EPSILON: 0.44869999946146477
LAST 100 EPISODE AVERAGE REWARD: 4.753682040419722
EPSILON: 0.4059802359226587
LAST 100 EPISODE AVERAGE REWARD: 3.685807139009064
EPSILON: 0.36732772934619257
LAST 100 EPISODE AVERAGE REWARD: 4.289442251896193
EPSILON: 0.33235524492954527
LAST 100 EPISODE AVERAGE REWARD: 2.837519318051739
EPSILON: 0.3007124156643058
LAST 100 EPISODE AVERAGE REWARD: 3.09060897137

In [22]:
Q_table[str(B@B)]

[10.434062, 0, -0.004347826086956522, 0]